# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [4]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# bring in weather csv
weatherfile = "cleaned_weather_data.csv"
we = pd.read_csv(weatherfile)
weather = pd.DataFrame(we).drop(columns=['Unnamed: 0'])

In [8]:
# filtering vacation params
hum_best = weather[weather['Temperature (Farenheit)'] <= 82]
hum_best2 = hum_best[hum_best['Temperature (Farenheit)'] >= 65]
hum_best3 = hum_best2[hum_best2['Humidity %'] <= 30]
hum_best3['lat_lngs'] = hum_best3['Latitude'].astype(str) + ", " + hum_best3['Longitude'].astype(str)

<ipython-input-8-fc56deb8b38e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hum_best3['lat_lngs'] = hum_best3['Latitude'].astype(str) + ", " + hum_best3['Longitude'].astype(str)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# set up a new column for table
hum_best3['Hotel Name'] = ""

<ipython-input-10-c5dbd1f6f08e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hum_best3['Hotel Name'] = ""


In [19]:
# find the closest hotel near each city
search = 'Hotel'
radius = 5000
t_type = 'lodging'

for index, row in hum_best3.iterrows():
    
    params= {
    "keyword":search,
    "radius":radius,
    "type":t_type,
    "key":g_key
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    params['location'] = row[9]
    
    print(f'Searching for The Closest Hotel to {row[0]}...')
    response = requests.get(base_url,params=params).json()
    
    results = response['results']
    try:
        print(f'The closest hotel is {results[0]["name"]}')
        
        hum_best3.loc[index, 'Hotel Name'] = results[0]["name"]

    except:
        print(f'No hotels near {row[0]}')

Searching for The Closest Hotel to Bilma...
No hotels near Bilma
Searching for The Closest Hotel to Kasane...
The closest hotel is Chobe Water Villas
Searching for The Closest Hotel to Ambon City...


C:\Users\Christina\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


The closest hotel is Swiss-Belhotel Ambon
Searching for The Closest Hotel to Kidal...
No hotels near Kidal
Searching for The Closest Hotel to Taoudenni...
No hotels near Taoudenni
Searching for The Closest Hotel to Umm Hājar...
No hotels near Umm Hājar
Searching for The Closest Hotel to Abū Zabad...
No hotels near Abū Zabad
Searching for The Closest Hotel to Sokolo...
No hotels near Sokolo
Searching for The Closest Hotel to Bikaner...
The closest hotel is The Laxmi Niwas Palace
Searching for The Closest Hotel to Miraflores...
The closest hotel is Hotel Mansión
Searching for The Closest Hotel to Kolokani...
The closest hotel is Maimouna sylla
Searching for The Closest Hotel to Tombouctou...
No hotels near Tombouctou


In [20]:
final_data = hum_best3[hum_best3['Hotel Name'] != ""]
final_data

,City Name,Latitude,Longitude,Temperature (Farenheit),Humidity %,Cloudiness %,Wind Speed (MPH),Country,Date,lat_lngs,Hotel Name
127,Kasane,-17.82,25.15,80.92,25,35,3.76,BW,1604960571,"-17.82, 25.15",Chobe Water Villas
140,Ambon City,-3.70,128.18,78.01,15,100,4.92,ID,1604960166,"-3.7, 128.18",Swiss-Belhotel Ambon
398,Bikaner,28.02,73.30,69.42,22,0,9.51,IN,1604960412,"28.02, 73.3",The Laxmi Niwas Palace
438,Miraflores,19.22,-98.81,75.22,27,40,4.70,MX,1604960636,"19.22, -98.81",Hotel Mansión
486,Kolokani,13.57,-8.03,77.54,18,0,6.87,ML,1604960648,"13.57, -8.03",Maimouna sylla


In [21]:
humidity = final_data['Humidity %'].astype(float)
max_hum = humidity.max()
locations = final_data[['Latitude','Longitude']]

In [23]:
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

humidity = final_data['Humidity %'].astype(float)
max_hum = humidity.max()
locations = final_data[['Latitude','Longitude']]

In [24]:
# Add heat map to globe

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_hum)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
final_data = [info_box_template.format(**row) for index, row in final_data.iterrows()]
locations = final_data[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
